<a href="https://colab.research.google.com/github/ghobtr/Colaboratory_Public/blob/main/p60n10ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoo_fin
!pip install requests_html

     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=2bc3050bf486b0d2a7e9cc5a89066be865f3af6cd7c830ac8bb38a61d5df1024
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.19.0-cp36-none-any.whl size=24582 sha256=4cee6482ad84a4dc9e174cc14baa2c9c590a52a0dc874ef57dfc019dce3ae408
  Stored in directory: /root/.cache/pip/wheels/c0/39/ea/e2fd678bd130953f5438470b8dfa529f00787e9b8b92b27467
Successfully built fake-useragent parse
ERROR: pyppeteer 0.2.5 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: importlib-metadata 3.4.0
    Uninstalling importlib-metadata-3.4.0:
      Successfully uninstalled importlib-metadat

In [ ]:
# univariate lstm example
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from yahoo_fin.stock_info import get_data
import plotly.graph_objects as go


# Global vars
stock_list = ["AKBNK.IS"]
for ticker in stock_list:

    split_percent = 0.95


    def get_price():
        df_stock = get_data(ticker, start_date=None, end_date=None, index_as_date=False, interval="1d")
        df_stock = df_stock.dropna()
        return df_stock

    # preparing independent and dependent features
    def prepare_data(timeseries_data, n_features):
        X, y = [], []
        for i in range(len(timeseries_data)):
            # find the end of this pattern
            end_ix = i + n_features
            # check if we are beyond the sequence
            if end_ix > len(timeseries_data) - 1:
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return np.array(X), np.array(y)


    df = get_price()
    close_data = df['close']
    # define input sequence

    n_steps = 60
    split = int(split_percent * len(close_data))

    close_train = close_data[:split]
    close_test = close_data[split:]
    close_train = np.array(close_train)
    close_test = np.array(close_test)

    # split into samples
    X_train, y_train = prepare_data(close_train, n_steps)
    X_test, y_test = prepare_data(close_test, n_steps)

    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)

    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    # define model
    model = Sequential([
            LSTM(120,input_shape=(X_train.shape[1],1),return_sequences=True),
          #     Dropout(0.3),
             LSTM(80,return_sequences=True),
        #     Dropout(0.3),    
            LSTM(60,return_sequences=False),
        #     Dense(20),
            Dense(1)
        ])

    model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])


    # fit model
    print("Model Train Start")
    model.fit(X_train, y_train, epochs=60, verbose=1)
    print("Model Train Finish")

    print("Model Test Prediction Start")
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
    y_predict_test = model.predict(X_test, verbose=1)

    y_predict_test = y_predict_test.reshape(-1)
    print("Model Test Prediction Finish")

    # Predicting For the next 10 data

    # demonstrate prediction for next 10 days
    x_input = y_test[-60:]
    temp_input = list(x_input)
    lst_output = []
    i = 0
    while (i < 10):

        if (len(temp_input) > 60):
            x_input = np.array(temp_input[1:])
            print("{} day input {}".format(i, x_input))
            # print(x_input)
            x_input = x_input.reshape((1, n_steps, n_features))
            # print(x_input)
            yhat = model.predict(x_input, verbose=0)
            print("{} day output {}".format(i, yhat))
            temp_input.append(yhat[0][0])
            temp_input = temp_input[1:]
            # print(temp_input)
            lst_output.append(yhat[0][0])
            i = i + 1
        else:
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            print(yhat[0])
            temp_input.append(yhat[0][0])
            lst_output.append(yhat[0][0])
            i = i + 1

    print(lst_output)

    # future_days = shift(lst_output, len(y_test), cval=np.NaN)

    # show plt
    trace1 = go.Scatter(
        # x=train_data.index.tolist(),
        y=y_test,
        mode='lines',
        name='Real Data'
    )
    trace2 = go.Scatter(
        # x=test_data.index.tolist(),
        y=y_predict_test,
        mode='lines',
        name='Predict Data'
    )
    trace3 = go.Scatter(
        # x=test_data.index.tolist(),
        y=lst_output,
        mode='lines',
        name='Future Data'
    )

    layout = go.Layout(
        title=ticker + "Stock",
        xaxis={'title': "Date"},
        yaxis={'title': "Close"}
    )

    fig2 = go.Figure(data=[trace1, trace2, trace3], layout=layout)
    fig2.write_html(ticker + 'p60n10.html', auto_open=False)


(4889, 60)
(201, 60)
(4889,)
(201,)
Model Train Start
Epoch 1/60
153/153 [==============================] - 11s 12ms/step - loss: 2.7862 - accuracy: 0.0000e+00
Epoch 2/60
153/153 [==============================] - 2s 12ms/step - loss: 0.2552 - accuracy: 0.0000e+00
Epoch 3/60
153/153 [==============================] - 2s 12ms/step - loss: 0.1452 - accuracy: 0.0000e+00
Epoch 4/60
153/153 [==============================] - 2s 11ms/step - loss: 0.1162 - accuracy: 0.0000e+00
Epoch 5/60
153/153 [==============================] - 2s 11ms/step - loss: 0.0813 - accuracy: 0.0000e+00
Epoch 6/60
153/153 [==============================] - 2s 11ms/step - loss: 0.0612 - accuracy: 0.0000e+00
Epoch 7/60
153/153 [==============================] - 2s 12ms/step - loss: 0.0619 - accuracy: 0.0000e+00
Epoch 8/60
153/153 [==============================] - 2s 11ms/step - loss: 0.0541 - accuracy: 0.0000e+00
Epoch 9/60
153/153 [==============================] - 2s 12ms/step - loss: 0.0489 - accuracy: 0.0000e+00
